In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData
from configs.ManualConfig import hparams

In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData
from configs.ManualConfig import hparams

### Construct necessary dictionaries
- already done and the results are in `data/dictionaries`

In [2]:
# constructBasicDict(attrs=['title'],path='/home/peitian_zhang/Data/MIND')

### View data

In [4]:
vocab, loaders = prepare(hparams, path="./data/MIND", pin_memory=False)

# for encoding news only
# vocab, loaders = prepare(hparams, news=True)

[2021-11-20 12:47:45,970] INFO (root) Hyper Parameters are
{'scale': 'small', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'learning_rate': 0.001, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cpu', 'attrs': ['title'], 'k': 0, 'coarse': None, 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'multiview': False, 'onehot': False, 'val_freq': 1, 'schedule': None}
[2021-11-20 12:47:45,972] INFO (root) preparing dataset...
[2021-11-20 12:48:04,878] INFO (torchtext.vocab) Loading vectors from .vector_cache\glove.840B.300d.txt
100%|█████████████████████████████████████████████████████████████████████▉| 2196016/2196017 [05:00<00:00, 7308.49it/s]
[2021-11-20 12:53:11,710] INFO (torchtext.vocab) Saving vectors to .vector_cache\glove.840B.300d.

In [5]:
# loader_train
a = next(iter(loaders[0]))
# loader_dev
b = next(iter(loaders[1]))

In [6]:
b

{'impression_index': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'user_index': tensor([[50001],
         [50001],
         [50001],
         [50001],
         [50001],
         [50001],
         [50001],
         [50001],
         [50001],
         [50001]], dtype=torch.int32),
 'cdd_id': tensor([[36180],
         [41328],
         [41034],
         [39776],
         [34983],
         [37322],
         [37327],
         [36307],
         [36185],
         [36349]], dtype=torch.int32),
 'candidate_title': tensor([[[  255,  9098,     5,  1290,  1482,     6,   314,  1087,   851,     9,
            3812,  7065,     1,     1,     1,     1,     1,     1,     1,     1]],
 
         [[  106,   834,   400,  6330,    75,  2025,   480,    10,    15,   157,
              17,   776,  3228,   104,  4912,     7,    15, 33313,    95,     5]],
 
         [[ 1064,  1273,  1253,    23,    75,     5,    83,    46,    91,  4247,
              15,   950,     1,     1,     1,     1,     1,     1,     1,     1]],
 
    

In [7]:
a['candidate_title_pad'].shape, b['candidate_title_pad'].shape

(torch.Size([10, 5, 20]), torch.Size([10, 1, 20]))

### Tailor Data to demo size

In [ ]:
# tailor 2000 impressions from MINDsmall_train to form MINDdemo_train
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [ ]:
analyse(hparams)

## The rest is for developing

In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict,tailorData
from configs.ManualConfig import hparams

import numpy as np
import re
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate
from torch.utils.data import DataLoader

In [2]:
from utils.MIND import MIND_impr

In [7]:
class MIND_impr(Dataset):
    """ Map Style Dataset for MIND, return each impression once

    Args:
        hparams(dict): pre-defined dictionary of hyper parameters
        news_file(str): path of news_file
        behaviors_file(str): path of behaviors_file
        shuffle(bool): whether to shuffle the order of impressions
    """

    def __init__(self, hparams, news_file, behaviors_file, shuffle_pos=False, validate=False):
        # initiate the whole iterator
        self.npratio = hparams['npratio']
        self.shuffle_pos = shuffle_pos

        self.news_file = news_file
        self.behaviors_file = behaviors_file
        self.col_spliter = '\t'
        self.batch_size = hparams['batch_size']
        self.title_size = hparams['title_size']
        self.abs_size = hparams['abs_size']
        self.his_size = hparams['his_size']

        self.onehot = hparams['onehot']
        self.k = hparams['k']

        # there are only two types of vocabulary
        self.vocab = getVocab('data/dictionaries/vocab_whole.pkl')

        self.nid2index = getId2idx(
            'data/dictionaries/nid2idx_{}_{}.json'.format(hparams['scale'], 'dev'))
        self.uid2index = getId2idx(
            'data/dictionaries/uid2idx_{}.json'.format(hparams['scale']))
        self.vert2onehot = getId2idx(
            'data/dictionaries/vert2onehot.json'
        )
        self.subvert2onehot = getId2idx(
            'data/dictionaries/subvert2onehot.json'
        )

        self.mode = 'dev'

        self.init_news()
        self.init_behaviors()

    def init_news(self):
        """
            init news information given news file, such as news_title_array.
        """

        # VERY IMPORTANT!!! FIXME
        # The nid2idx dictionary must follow the original order of news in news.tsv

        titles = [[1]*self.title_size]
        title_pad = [[self.title_size]]

        with open(self.news_file, "r", encoding='utf-8') as rd:
            for idx in rd:
                nid, vert, subvert, title, ab, url, _, _ = idx.strip("\n").split(self.col_spliter)

                title_token = tokenize(title, self.vocab)
                titles.append(title_token[:self.title_size] + [1] * (self.title_size - len(title_token)))
                title_pad.append([max(self.title_size - len(title_token), 0)])

        # self.titles = titles
        self.news_title_array = np.asarray(titles)
        self.title_pad = np.asarray(title_pad)

    def init_behaviors(self):
        """
            init behavior logs given behaviors file.
        """
        # list of list of history news index
        self.histories = []
        # list of user index
        self.uindexes = []
        # list of list of history padding length
        self.his_pad = []
        # list of impression indexes
        # self.impr_indexes = []

        # list of every candidate news index along with its impression index and label
        self.imprs = []

        with open(self.behaviors_file, "r", encoding='utf-8') as rd:
            for idx in rd:
                impr_index, uid, time, history, impr = idx.strip("\n").split(self.col_spliter)
                impr_index = int(impr_index) - 1

                history = [self.nid2index[i] for i in history.split()]
                # tailor user's history or pad 0
                history = history[:self.his_size] + [0] * (self.his_size - len(history))
                impr_news = [self.nid2index[i.split("-")[0]] for i in impr.split()]
                labels = [int(i.split("-")[1]) for i in impr.split()]
                # user will always in uid2index
                uindex = self.uid2index[uid]

                # store every impression
                self.imprs.append((impr_index, impr_news[0], labels[0]))

                # 1 impression correspond to 1 of each of the following properties
                self.histories.append(history)
                self.uindexes.append(uindex)

    def __len__(self):
        """
            return length of the whole dataset
        """
        return len(self.imprs)

    def __getitem__(self,index):
        """ return data
        Args:
            index: the index for stored impression

        Returns:
            back_dic: dictionary of data slice
        """

        impr = self.imprs[index] # (impression_index, news_index)
        impr_index = impr[0]
        impr_news = impr[1]


        user_index = [self.uindexes[impr_index]]

        cdd_ids = [impr_news]

        his_ids = self.histories[impr_index]

        user_index = [self.uindexes[impr_index]]
        label = impr[2]

        candidate_title_index = [self.news_title_array[impr_news]]
        clicked_title_index = self.news_title_array[his_ids]
        back_dic = {
            "impression_index": impr_index + 1,
            "user_index": np.asarray(user_index),
            'cdd_id': np.asarray(cdd_ids),
            "candidate_title": np.asarray(candidate_title_index),
            'his_id': np.asarray(his_ids),
            "clicked_title": clicked_title_index,
            "labels": np.asarray([label])
        }

        return back_dic

In [8]:
hparams['mode'] = 'dev'
# hparams['onehot'] = True
path='/home/peitian_zhang/Data/MIND'
news_file = path+'/MIND'+hparams['scale']+'_{}/news.tsv'.format(hparams['mode'])
behavior_file = path+'/MIND' + hparams['scale']+'_{}/behaviors.tsv'.format(hparams['mode'])
dataset = MIND_impr(hparams=hparams, news_file=news_file,behaviors_file=behavior_file)

In [15]:
loader = DataLoader(dataset, batch_size=5, collate_fn=my_collate)
next(iter(loader))

{'impression_index': [1, 2, 3, 4, 5],
 'user_index': tensor([[1929],
         [1930],
         [1931],
         [1932],
         [1933]]),
 'cdd_id': tensor([[36180],
         [35201],
         [39669],
         [  438],
         [39227]]),
 'candidate_title': tensor([[[  255,  9098,     5,  1290,  1482,     6,   314,  1087,   851,     9,
            3812,  7065,     1,     1,     1,     1,     1,     1,     1,     1]],
 
         [[  594,    81,   264,   229,   561,    29,  2688,     1,     1,     1,
               1,     1,     1,     1,     1,     1,     1,     1,     1,     1]],
 
         [[  206,   634,   391,   213,   240,     7,   732,    10,   638,  1702,
              67,   132,     1,     1,     1,     1,     1,     1,     1,     1]],
 
         [[  106, 47085,    18,  1206,  1216,   106,  1965,  1514,    42,   138,
            2443,  1223,    16, 17795,    43,   366,   144,  1622,   158,     1]],
 
         [[ 1479,  2277,    17,  2880,  1404,  6455,  3228,     8,  2377,  1